In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import catboost as cat_
import seaborn as sns
import lightgbm as lgb
import re
import timeit
import random
random.seed(3)

In [2]:
# WORK ON OUTLINES USING LOG

# Import the train data
train = pd.read_csv('datasets/SUPCOM_Train.csv')

# Import the test data
test = pd.read_csv('datasets/SUPCOM_Test.csv')

Submission = pd.read_csv('datasets/SUPCOM_SampleSubmission.csv')
variables = pd.read_csv('datasets/VariableDescription.csv')

In [3]:
train.head()

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,...,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,id
0,44,6210,C,99,0,3707,4,2,2,2,...,16.639203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id17437
1,401,14383,M,22,0,6105,4,2,2,2,...,17.827703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id5086
2,243,11555,M,23,0,5751,4,2,2,2,...,15.473503,25387000.0,0.0,0.0,4571280.0,NaN,NaN,NaN,NaN,train_id1443
3,72,6175,C,99,0,4402,4,2,2,2,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id15469
4,42,1417,M,23,0,6302,4,2,2,2,...,14.286244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id14368


In [4]:
target = train.target.values
print(target.shape)

(21295,)


In [5]:
# Combine train and test data for easy preprocessing
ntrain = train.shape[0]
ntest = test.shape[0]
data = pd.concat((train, test),sort=False)
print(data.shape)
data.head()

(28812, 121)


,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,...,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,id
0,44,6210,C,99,0,3707,4,2,2,2,...,16.639203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id17437
1,401,14383,M,22,0,6105,4,2,2,2,...,17.827703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id5086
2,243,11555,M,23,0,5751,4,2,2,2,...,15.473503,25387000.0,0.0,0.0,4571280.0,NaN,NaN,NaN,NaN,train_id1443
3,72,6175,C,99,0,4402,4,2,2,2,...,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id15469
4,42,1417,M,23,0,6302,4,2,2,2,...,14.286244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train_id14368


In [6]:
variables.head(60)

,Libellé,Intitulé,Title
0,Contribuable,NaN,Rate payer
1,BCT_CODBUR,Code Bureau de contrôle des impôts,Code Tax Control Office
2,CTR_MATFIS,ID Contribuable,Rate payer ID
3,CTR_CATEGO_x,Catégorie Contribuable,Rate payer category
4,CTR_CODTVA,Code TVA,Code TVA
5,FJU_CODFJU,Code Forme Juridique,Code Legal Form
6,CTR_CESSAT,"Code Cessation (0 : Actif, 1 : en cessation)","Cessation Code (0: Active, 1: Inactive)"
7,ACT_CODACT,Code Activité,Activity Code
8,CTR_OBLDIR,Obligation,Obligation
9,CTR_OBLACP,Obligation Acompte Provisionnel,Provisional Down Payment Obligation


In [7]:
variables.tail(60)

,Libellé,Intitulé,Title
97,SND_MNTDRC_E,Montant Droit de Consommation à l'Exportation,Amount of Export Consumption Right
98,SND_MNTAVA_E,Montant Avance à l'Exportation,Amount Advance for Export
99,SND_MNTTAX_E,Montant Autres Taxes Payés à l'Exportation,Amount Other Taxes Paid for Exportation\n
100,SND_MNTPAY_E,Montant Total Payé à l'Exportation,Montant Total Payé à l'Exportation
101,SND_MNTAIR_E,NaN,NaN
102,SND_MNTPRD_A,Montant Produit autres régimes,Amount Product other plans
103,SND_MNTTVA_A,Montant TVA autres régimes,VAT amount other schemes
104,SND_MNTDRC_A,Montant Droit de Consommation autres régimes,Amount Consumer Right other plans
105,SND_MNTAVA_A,Montant Avance autres régimes,Amount Advance other plans
106,SND_MNTTAX_A,Montant Autres Taxes Payés autres régimes,Amount Other Taxes Paid other plans


In [8]:
len(data.columns)

121

In [9]:
data.describe()

,BCT_CODBUR,CTR_MATFIS,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,...,RES_ANNIMP,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C
count,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,...,28812.000000,21295.000000,2.479000e+03,2.479000e+03,2.479000e+03,2.479000e+03,2.865000e+03,2.865000e+03,2.865000e+03,2.865000e+03
mean,215.855546,7816.644315,59.239969,0.049181,4825.815945,4.453700,1.960121,1.976572,1.927669,1.948702,...,2014.516243,11.780084,8.243376e+09,5.433622e+06,6.729353e+05,4.461729e+08,8.660866e+08,1.359557e+06,2.469147e+04,1.414242e+08
std,163.139993,4489.238442,38.531542,0.230543,1165.478787,1.204495,0.195679,0.155336,0.263688,0.223267,...,1.336841,7.085772,1.175995e+11,1.793214e+08,1.933025e+07,4.105284e+09,4.265450e+09,1.535796e+07,6.111973e+05,7.400228e+08
min,9.000000,1.000000,10.000000,0.000000,1002.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,2013.000000,0.000000,9.290000e+02,0.000000e+00,0.000000e+00,0.000000e+00,4.910000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,92.000000,3949.000000,22.000000,0.000000,4203.000000,4.000000,2.000000,2.000000,2.000000,2.000000,...,2013.000000,0.000000,2.993120e+07,0.000000e+00,0.000000e+00,4.323436e+06,1.957640e+07,0.000000e+00,0.000000e+00,3.021017e+06
50%,192.000000,7851.500000,23.000000,0.000000,4417.000000,4.000000,2.000000,2.000000,2.000000,2.000000,...,2014.000000,14.981121,1.891549e+08,0.000000e+00,0.000000e+00,2.806525e+07,5.722739e+07,0.000000e+00,0.000000e+00,8.882093e+06
75%,254.000000,11710.250000,99.000000,0.000000,6102.000000,4.000000,2.000000,2.000000,2.000000,2.000000,...,2015.000000,16.625623,9.674418e+08,0.000000e+00,0.000000e+00,1.459402e+08,2.754562e+08,0.000000e+00,0.000000e+00,4.449364e+07
max,700.000000,15603.000000,99.000000,2.000000,8999.000000,9.000000,2.000000,5.000000,5.000000,3.000000,...,2018.000000,23.591334,3.414510e+12,8.897885e+09,8.212377e+08,1.608780e+11,8.394391e+10,4.044067e+08,2.614500e+07,1.510990e+10


## Data quality check

In [10]:
# This function seperate the columns into the various data types
def data_types(dataframe):
  int_col = []
  continuous_col = []
  cat_col = []
  for column in train.columns:
    if train[column].dtypes == 'int64':
      int_col.append(column)
    elif train[column].dtypes == 'float':
      continuous_col.append(column)
    else:
      cat_col.append(column)
  return int_col, continuous_col, cat_col

In [11]:
int_col, continuous_col, cat_col = data_types(train)

In [12]:
print('Ordinal Variables', len(int_col))
print('Continuous Variables',len(continuous_col))
print('Categorical Variables', len(cat_col))

Ordinal Variables 16
Continuous Variables 103
Categorical Variables 2


In [13]:
print(int_col)

['BCT_CODBUR', 'CTR_MATFIS', 'FJU_CODFJU', 'CTR_CESSAT', 'ACT_CODACT', 'CTR_OBLDIR', 'CTR_OBLACP', 'CTR_OBLRES', 'CTR_OBLFOP', 'CTR_OBLTFP', 'CTR_OBLDCO', 'CTR_OBLTVA', 'CTR_OBLTCL', 'CTR_RATISS', 'EXE_EXERCI', 'RES_ANNIMP']


### Understanding the int64 data type variables

In [14]:
data[int_col].describe()

,BCT_CODBUR,CTR_MATFIS,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OBLTCL,CTR_RATISS,EXE_EXERCI,RES_ANNIMP
count,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000,28812.000000
mean,215.855546,7816.644315,59.239969,0.049181,4825.815945,4.453700,1.960121,1.976572,1.927669,1.948702,1.005206,2.039289,2.983236,0.022144,2014.516243,2014.516243
std,163.139993,4489.238442,38.531542,0.230543,1165.478787,1.204495,0.195679,0.155336,0.263688,0.223267,0.071967,0.194285,0.182054,0.209280,1.336841,1.336841
min,9.000000,1.000000,10.000000,0.000000,1002.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,0.000000,2013.000000,2013.000000
25%,92.000000,3949.000000,22.000000,0.000000,4203.000000,4.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,3.000000,0.000000,2013.000000,2013.000000
50%,192.000000,7851.500000,23.000000,0.000000,4417.000000,4.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,3.000000,0.000000,2014.000000,2014.000000
75%,254.000000,11710.250000,99.000000,0.000000,6102.000000,4.000000,2.000000,2.000000,2.000000,2.000000,1.000000,2.000000,3.000000,0.000000,2015.000000,2015.000000
max,700.000000,15603.000000,99.000000,2.000000,8999.000000,9.000000,2.000000,5.000000,5.000000,3.000000,2.000000,3.000000,3.000000,2.000000,2018.000000,2018.000000


In [15]:
data[int_col].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28812 entries, 0 to 7516
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   BCT_CODBUR  28812 non-null  int64
 1   CTR_MATFIS  28812 non-null  int64
 2   FJU_CODFJU  28812 non-null  int64
 3   CTR_CESSAT  28812 non-null  int64
 4   ACT_CODACT  28812 non-null  int64
 5   CTR_OBLDIR  28812 non-null  int64
 6   CTR_OBLACP  28812 non-null  int64
 7   CTR_OBLRES  28812 non-null  int64
 8   CTR_OBLFOP  28812 non-null  int64
 9   CTR_OBLTFP  28812 non-null  int64
 10  CTR_OBLDCO  28812 non-null  int64
 11  CTR_OBLTVA  28812 non-null  int64
 12  CTR_OBLTCL  28812 non-null  int64
 13  CTR_RATISS  28812 non-null  int64
 14  EXE_EXERCI  28812 non-null  int64
 15  RES_ANNIMP  28812 non-null  int64
dtypes: int64(16)
memory usage: 3.7 MB


In [16]:
data[cat_col].head()

,CTR_CATEGO_X,id
0,C,train_id17437
1,M,train_id5086
2,M,train_id1443
3,C,train_id15469
4,M,train_id14368


In [17]:
#data = data.drop("id", axis=1)

In [18]:
data.CTR_CATEGO_X.unique()

array(['C', 'M', 'P', 'N'], dtype=object)

In [19]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
LE = LabelEncoder()

In [20]:
data[continuous_col].describe()

,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,CTR_OBLAUT,CTR_OBLASS,CTR_ODTIMB,CTR_OBLTHO,CTR_OBLDLI,CTR_OBLTVI,TVA_CHAFF6,...,AX5_MNTMAR,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C
count,28801.000000,28801.000000,28801.000000,28801.000000,28801.000000,28801.000000,28801.000000,28801.000000,28801.000000,2.656700e+04,...,2.184300e+04,21295.000000,2.479000e+03,2.479000e+03,2.479000e+03,2.479000e+03,2.865000e+03,2.865000e+03,2.865000e+03,2.865000e+03
mean,1.091872,1.003368,1.017222,1.019756,1.000069,1.475122,1.012743,1.037568,1.002118,1.773663e+08,...,1.030642e+09,11.780084,8.243376e+09,5.433622e+06,6.729353e+05,4.461729e+08,8.660866e+08,1.359557e+06,2.469147e+04,1.414242e+08
std,0.288850,0.057937,0.130099,0.139164,0.008333,0.499389,0.112164,0.190153,0.045974,2.172927e+09,...,8.053263e+09,7.085772,1.175995e+11,1.793214e+08,1.933025e+07,4.105284e+09,4.265450e+09,1.535796e+07,6.111973e+05,7.400228e+08
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,...,0.000000e+00,0.000000,9.290000e+02,0.000000e+00,0.000000e+00,0.000000e+00,4.910000e+02,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,...,0.000000e+00,0.000000,2.993120e+07,0.000000e+00,0.000000e+00,4.323436e+06,1.957640e+07,0.000000e+00,0.000000e+00,3.021017e+06
50%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,...,4.857380e+06,14.981121,1.891549e+08,0.000000e+00,0.000000e+00,2.806525e+07,5.722739e+07,0.000000e+00,0.000000e+00,8.882093e+06
75%,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,0.000000e+00,...,1.600982e+08,16.625623,9.674418e+08,0.000000e+00,0.000000e+00,1.459402e+08,2.754562e+08,0.000000e+00,0.000000e+00,4.449364e+07
max,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.060280e+11,...,6.939760e+11,23.591334,3.414510e+12,8.897885e+09,8.212377e+08,1.608780e+11,8.394391e+10,4.044067e+08,2.614500e+07,1.510990e+10


In [21]:
# All the values in some variables are only zeros. 
col_only_zeros = []
for column in continuous_col:
    if data[column].max() == 0 and data[column].min() == 0:
        col_only_zeros.append(column)

print(len(col_only_zeros))
print(col_only_zeros)

11
['TVA_CAF125', 'TVA_CHAF36', 'TVA_MTORDTAX', 'TVA_MTDJRTAX', 'TVA_MTRESTAX', 'SND_MNTAVA_I', 'SND_MNTTVA_E', 'SND_MNTDRC_E', 'SND_MNTAVA_E', 'SND_MNTAIR_E', 'SND_MNTAVA_A']


In [22]:
print(cat_col)

['CTR_CATEGO_X', 'id']


In [23]:
int_col_use = ['CTR_OBLDIR', 'CTR_OBLACP', 'CTR_OBLRES', 'CTR_OBLFOP', 'CTR_OBLTFP', 'CTR_OBLDCO', 'CTR_OBLTVA', 'CTR_OBLTCL', 'CTR_RATISS']
int_col_drop = ['BCT_CODBUR', 'CTR_MATFIS', 'FJU_CODFJU', 'CTR_CESSAT', 'ACT_CODACT', 'EXE_EXERCI', 'RES_ANNIMP']

In [24]:
redundant_col = int_col_drop + col_only_zeros + ['id']
print(len(redundant_col),redundant_col)

19 ['BCT_CODBUR', 'CTR_MATFIS', 'FJU_CODFJU', 'CTR_CESSAT', 'ACT_CODACT', 'EXE_EXERCI', 'RES_ANNIMP', 'TVA_CAF125', 'TVA_CHAF36', 'TVA_MTORDTAX', 'TVA_MTDJRTAX', 'TVA_MTRESTAX', 'SND_MNTAVA_I', 'SND_MNTTVA_E', 'SND_MNTDRC_E', 'SND_MNTAVA_E', 'SND_MNTAIR_E', 'SND_MNTAVA_A', 'id']


In [25]:
# Function to handle missing values
def missing_values(data, threshold = 30.0):
    drop_columns = []
    keep_columns = []
    columns = data.columns 
    for column in columns:
        missing_val_percent = (100 * data[column].isna().sum() / len(data))
        if missing_val_percent >= threshold:
            drop_columns.append(column)
        else:
            keep_columns.append(column)
    return (drop_columns, keep_columns)

In [26]:
# Columns to drop and those to keep due to missing values
drop_columns , keep_columns = missing_values(data)

print('Number of columns with more than 30% missing values:', len(drop_columns))
print('Number of columns to keep:', len(keep_columns))

Number of columns with more than 30% missing values: 30
Number of columns to keep: 91


In [27]:
# Columns to keep 
data[keep_columns].head()

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,...,TVA_CA132018,TVA_CA192018,AX1_BRTIMP,AX2_HONORA,AX2_LOYERS,AX3_REVCAP,AX5_MNTMAR,RES_ANNIMP,target,id
0,44,6210,C,99,0,3707,4,2,2,2,...,0.0,0.0,0.0,2950000.0,0.0,0.0,602769328.0,2014,16.639203,train_id17437
1,401,14383,M,22,0,6105,4,2,2,2,...,0.0,0.0,0.0,203353885.0,0.0,0.0,14785200.0,2013,17.827703,train_id5086
2,243,11555,M,23,0,5751,4,2,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,10792800.0,2013,15.473503,train_id1443
3,72,6175,C,99,0,4402,4,2,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2017,0.000000,train_id15469
4,42,1417,M,23,0,6302,4,2,2,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2014,14.286244,train_id14368


In [28]:
# Drop redundant columns
data2 = data.drop(redundant_col, axis = 1)
print(data2.shape)
data2.head()

(28812, 102)


,CTR_CATEGO_X,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,...,AX5_MNTMAR,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C
0,C,4,2,2,2,2,1,2,1.0,1.0,...,602769328.0,16.639203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M,4,2,2,2,2,1,2,1.0,1.0,...,14785200.0,17.827703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M,4,2,2,2,2,1,2,2.0,1.0,...,10792800.0,15.473503,25387000.0,0.0,0.0,4571280.0,NaN,NaN,NaN,NaN
3,C,4,2,2,2,2,1,2,1.0,1.0,...,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M,4,2,2,2,2,1,2,1.0,1.0,...,0.0,14.286244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
data2.CTR_CATEGO_X.unique()

array(['C', 'M', 'P', 'N'], dtype=object)

In [30]:
data2.head()

,CTR_CATEGO_X,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,...,AX5_MNTMAR,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C
0,C,4,2,2,2,2,1,2,1.0,1.0,...,602769328.0,16.639203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,M,4,2,2,2,2,1,2,1.0,1.0,...,14785200.0,17.827703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,M,4,2,2,2,2,1,2,2.0,1.0,...,10792800.0,15.473503,25387000.0,0.0,0.0,4571280.0,NaN,NaN,NaN,NaN
3,C,4,2,2,2,2,1,2,1.0,1.0,...,0.0,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,M,4,2,2,2,2,1,2,1.0,1.0,...,0.0,14.286244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
# Convert the Categorical columns to dummies
data_dumi = pd.get_dummies(data2)

In [32]:
data_dumi.tail()

,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,CTR_OBLDCO,CTR_OBLTVA,CTR_OFODEC,CTR_OFODEP,CTR_OFODET,...,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,CTR_CATEGO_X_C,CTR_CATEGO_X_M,CTR_CATEGO_X_N,CTR_CATEGO_X_P
7512,4,2,2,2,2,1,2,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,1
7513,4,2,2,2,2,1,2,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0
7514,4,2,2,2,2,1,2,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0
7515,4,2,2,2,2,1,2,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0
7516,4,2,2,2,2,1,2,1.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0


In [33]:
data.drop('id', inplace=True, axis=1)

In [34]:
data.head()

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,...,RES_ANNIMP,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C
0,44,6210,C,99,0,3707,4,2,2,2,...,2014,16.639203,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,401,14383,M,22,0,6105,4,2,2,2,...,2013,17.827703,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,243,11555,M,23,0,5751,4,2,2,2,...,2013,15.473503,25387000.0,0.0,0.0,4571280.0,NaN,NaN,NaN,NaN
3,72,6175,C,99,0,4402,4,2,2,2,...,2017,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,42,1417,M,23,0,6302,4,2,2,2,...,2014,14.286244,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
data.tail()

,BCT_CODBUR,CTR_MATFIS,CTR_CATEGO_X,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,...,RES_ANNIMP,target,FAC_MNTPRI_F,FAC_MFODEC_F,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C
7512,193,1162,P,99,0,6307,4,2,2,2,...,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7513,68,5572,M,22,0,3802,4,2,2,2,...,2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7514,173,2943,M,22,0,4221,4,2,2,2,...,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7515,65,6583,M,22,0,4101,4,2,2,2,...,2013,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7516,16,5883,M,22,0,2930,4,2,2,2,...,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
data = pd.get_dummies(data)

In [37]:
data.head()

,BCT_CODBUR,CTR_MATFIS,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,...,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,CTR_CATEGO_X_C,CTR_CATEGO_X_M,CTR_CATEGO_X_N,CTR_CATEGO_X_P
0,44,6210,99,0,3707,4,2,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
1,401,14383,22,0,6105,4,2,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0
2,243,11555,23,0,5751,4,2,2,2,2,...,0.0,4571280.0,NaN,NaN,NaN,NaN,0,1,0,0
3,72,6175,99,0,4402,4,2,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0,0
4,42,1417,23,0,6302,4,2,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0


In [38]:
data.shape

(28812, 123)

# SCALING THE DATA 

In [39]:
from sklearn.preprocessing import MaxAbsScaler
scaler = MaxAbsScaler()
data_sc_raw = scaler.fit_transform(data)
data_sc_raw = pd.DataFrame(data_sc_raw, columns = data.columns)

In [40]:
data_sc_raw.head(4)

,BCT_CODBUR,CTR_MATFIS,FJU_CODFJU,CTR_CESSAT,ACT_CODACT,CTR_OBLDIR,CTR_OBLACP,CTR_OBLRES,CTR_OBLFOP,CTR_OBLTFP,...,FAC_MNTDCO_F,FAC_MNTTVA_F,FAC_MNTPRI_C,FAC_MFODEC_C,FAC_MNTDCO_C,FAC_MNTTVA_C,CTR_CATEGO_X_C,CTR_CATEGO_X_M,CTR_CATEGO_X_N,CTR_CATEGO_X_P
0,0.062857,0.398000,1.000000,0.0,0.411935,0.444444,1.0,0.4,0.4,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0
1,0.572857,0.921810,0.222222,0.0,0.678409,0.444444,1.0,0.4,0.4,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0
2,0.347143,0.740563,0.232323,0.0,0.639071,0.444444,1.0,0.4,0.4,0.666667,...,0.0,0.000028,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0
3,0.102857,0.395757,1.000000,0.0,0.489165,0.444444,1.0,0.4,0.4,0.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0


In [41]:
#Train and test datasets
train = data_sc_raw[:ntrain].copy()
test = data_sc_raw[ntrain:].copy()
test = test.reset_index(drop=True)

In [42]:
print('train:',train.shape)
print('test:', test.shape)

train: (21295, 123)
test: (7517, 123)


In [43]:
# lightgbm for regression
from numpy import mean
from numpy import std
from lightgbm import LGBMRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot

# evaluate the model
model = LGBMRegressor()
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, train, target, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('LGBM MSE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# fit the model on the whole dataset
model_lg = LGBMRegressor()
model_lg.fit(train, target)

LGBM MSE: -0.025 (0.013)


LGBMRegressor()

In [49]:
model_lg = model_lg.predict(test)

In [44]:
# catboost for regression
from numpy import mean
from numpy import std
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot
# evaluate the model
model = CatBoostRegressor(verbose=0, n_estimators=100)
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, train, target, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('CAT MSE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# fit the model on the whole dataset
model_cat = CatBoostRegressor(verbose=0, n_estimators=100)
model_cat.fit(train, target)

CAT MSE: -0.032 (0.010)


In [50]:
model_cat = model_cat.predict(test)

In [45]:
# xgboost for regression
from numpy import mean
from numpy import std
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from matplotlib import pyplot

# evaluate the model
model = XGBRegressor(objective='reg:squarederror')
cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, train, target, scoring='neg_mean_squared_error', cv=cv, n_jobs=-1, error_score='raise')
print('XGBM MSE: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# fit the model on the whole dataset
model_xg = XGBRegressor(objective='reg:squarederror')
model_xg.fit(train, target)

XGBM MSE: -0.000 (0.000)


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=4, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [51]:
model_xg = model_xg.predict(test)

In [46]:
#pip install mlxtend  

In [47]:
from sklearn.linear_model import RidgeCV
from sklearn.svm import LinearSVR
#from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import StackingRegressor

In [48]:
# make a prediction with a stacking mlxtend
from sklearn.linear_model import LinearRegression
#import mlxtend
#from sklearn.ensemble import StackingRegressor
from mlxtend.regressor import StackingRegressor

# define meta learner model
lin_reg = LinearRegression()
# define the stacking ensemble
model = StackingRegressor(regressors=[model_lg, model_cat, model_xg], meta_regressor=lin_reg)
# fit the model on all available data
model.fit(train, target)
stack_result = model.predict(test)


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
#conda install mlxtend

# SUBMISSION

In [ ]:
Submission['target'] = stack_result #model_lg.predict(test) 
file_name = 'old-stack3-raw.csv'
Submission.to_csv(file_name ,index=False)

In [52]:
Submission['target'] = model_lg #model_lg.predict(test) 
file_name = 'lightGBM_old-raw.csv'
Submission.to_csv(file_name ,index=False)

In [53]:
Submission['target'] = model_cat #model_lg.predict(test) 
file_name = 'Catboost_old-raw.csv'
Submission.to_csv(file_name ,index=False)

In [55]:
Submission['target'] = model_xg #model_lg.predict(test) 
file_name = 'XGBoost_old-raw.csv'
Submission.to_csv(file_name ,index=False)